In [13]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api = os.getenv('GROQ_API_KEY')

In [30]:
llm = ChatGroq(model="llama-3.1-70b-versatile",groq_api_key =groq_api) 

In [29]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43133")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Machine Learning Engineer AI/ML - Consumer Marketplace (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsLead Machine Learning Engineer

In [31]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [32]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

In [33]:
json_res

{'role': 'Lead Machine Learning Engineer AI/ML - Consumer Marketplace',
 'experience': '5+ years of experience in enterprise environment with a combination of technology and team leadership responsibilities',
 'skills': ['Python',
  'Spark',
  'Java',
  'mlFlow',
  'Airflow',
  'Docker',
  'AWS/GCP',
  'S3',
  'Dynamo'],
 'description': 'Apply machine learning, collaborative filtering, NLP, and deep learning methods to massive data. Provide technical leadership, establishing standards and principles for the team to follow. Support investigation of new software packages/tools, APIs, and algorithms to deliver quality analytics and machine learning at scale.'}

In [34]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [35]:
job = json_res
job['skills']

['Python',
 'Spark',
 'Java',
 'mlFlow',
 'Airflow',
 'Docker',
 'AWS/GCP',
 'S3',
 'Dynamo']

In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\Welcome\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:37<00:00, 2.21MiB/s]


In [36]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-port

In [37]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nilofer, a business development executive at XXX. XXX is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XXX 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XXX's portfolio: {link_list}
        Remember you are Nilofer, BDE at XXX. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Solutions for Your Consumer Marketplace

Dear Hiring Manager,

I came across the job posting for a Lead Machine Learning Engineer AI/ML - Consumer Marketplace and was impressed by the role's focus on harnessing machine learning, collaborative filtering, NLP, and deep learning methods to drive business growth. As a Business Development Executive at XXX, I'd like to highlight how our company can support your organization in achieving these goals.

With our expertise in AI and software consulting, we empower businesses to integrate automated tools seamlessly, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a proven track record of delivering high-quality solutions, leveraging technologies such as Python, Spark, Java, mlFlow, Airflow, Docker, AWS/GCP, S3, and Dynamo.

Our portfolio showcases our capabilities in machine learning and related technologies. Some notable examples include:

-